**Bu notebookta train.py için huggingface de bulunan bir türkçe şiirler veri setini indirip karakter düzeyinde tokenize edip train.py için hazır hale getiriyoruz.**

In [1]:
import json
from tqdm import tqdm
import numpy as np
from datasets import load_dataset 

In [4]:
dataset = load_dataset('beratcmn/turkish-poems-cleaned')

unwanted_columns = ['title', 'rating', 'id', 'poet']
split_dataset = dataset['train'].train_test_split(test_size=0.1, seed=1337)
split_dataset = split_dataset.remove_columns(unwanted_columns)
split_dataset['val'] = split_dataset.pop('test') # test splitinin ismini val olarak değişiyoruz
split_dataset

DatasetDict({
    train: Dataset({
        features: ['poem'],
        num_rows: 4464
    })
    val: Dataset({
        features: ['poem'],
        num_rows: 497
    })
})

In [5]:
# character level tokenizer oluşturmak için tüm veri setindeki ayrı karakterleri topluyoruz ve sözlük oluşturuyoruz
def create_vocab(dataset):
    vocab = set()
    for example in dataset:
        vocab.update(example['poem'])
    vocab = sorted(vocab)
    char_to_id = {char: idx for idx, char in enumerate(vocab, start=1)}  # 0 -> bilinmeyen token olarak reserve.
    char_to_id['<cs>'] = len(char_to_id) + 1  # vocab'ın sonuna end of text tokeni ekliyoruz (cs -> cümle sonu)
    char_to_id['<bk>'] = 0 # bk -> bilinmeyen karakter
    return char_to_id

# sözlüğü oluşturduktan sonra kaydetmek için.
def save_vocab_as_json(vocab, filename="vocab.json"):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(vocab, f, ensure_ascii=False, indent=4)
    print(f"Karakter sözlüğü {filename} olarak kaydedildi.")

In [6]:
vocab = create_vocab(split_dataset['train'])
eot_token = vocab['<cs>']
save_vocab_as_json(vocab, 'vocab.json')

Karakter sözlüğü vocab.json olarak kaydedildi.


In [7]:
print(f'Toplam sözlük boyutu : {len(vocab)}')

Toplam sözlük boyutu : 141


In [8]:
# her karakteri denk geldiği id'ye çeviriyoruz
def process(example):
    ids = [vocab.get(char, 0) for char in example['poem']] 
    ids.append(eot_token)  # her şiirin sonuna modele context verebilmek için end-of-text tokeni ekliyoruz
    return {'ids': ids, 'len': len(ids)}

In [9]:
tokenized = split_dataset.map(
    process,
    remove_columns=['poem'],
    desc="Tokenize ediliyor.."
)

Tokenize ediliyor..:   0%|          | 0/4464 [00:00<?, ? examples/s]

Tokenize ediliyor..:   0%|          | 0/497 [00:00<?, ? examples/s]

In [10]:
for split, dset in tokenized.items():
    arr_len = np.sum(dset['len'], dtype=np.uint64)
    filename = f'{split}.bin'
    dtype = np.uint8 # sözlük boyutu oldukça küçük olduğu için uint8 kullanabiliriz (2**8 > 139)
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
    total_batches = 256

    idx = 0
    for batch_idx in tqdm(range(total_batches), desc=f'{filename} oluşturuluyor...'):
        batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
        arr_batch = np.concatenate(batch['ids'])
        arr[idx:idx + len(arr_batch)] = arr_batch
        idx += len(arr_batch)
    arr.flush()

val.bin oluşturuluyor...: 100%|██████████| 256/256 [00:00<00:00, 1034.67it/s]
